Made by 
    Luca Sturaro (Polito s320062)
In collaboration with 
    Gabriele Tomatis (Polito, s3xxxxx)

In [43]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np

In [44]:
PROBLEM_SIZE = 500
NUM_SETS = 1000
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

In [45]:
if PROBLEM_SIZE <= 100:
    print("Sets:")
    for num, element in enumerate(SETS):
        print(f'{num}:\t', end='')
        for value in element:
            if value:
                print('1 ', end='')
            else:
                print('0 ', end='')
        print()

max_solution = [0] * PROBLEM_SIZE
for i in range(PROBLEM_SIZE):
    for element in SETS:
        if element[i]:
            max_solution[i] += element[i]

if 0 in max_solution:
    print('Problem not solvable')
    raise Exception('Problem is not solvable')
else:
    print('Problem solvable')
# print(max_solution)

Problem solvable


In [46]:
def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))


def distance(state):
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))

def weight(state):
    # f = g + h
    g = len(state.taken)
    progress = [False] * PROBLEM_SIZE
    for set in state.taken:
        list_set = SETS[set].tolist()
        for i in range(PROBLEM_SIZE):
            progress[i] = progress[i] or list_set[i]
    h = PROBLEM_SIZE - sum(progress)
    return g + h

In [47]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((0, state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((weight(new_state), new_state))
    _, current_state = frontier.get()

print(f'Solved in {counter:,} steps ({len(current_state.taken)} tiles)')
print(f'Solution:     Taken: {current_state.taken}')

Solved in 8 steps (8 tiles)
Solution:     Taken: {36, 485, 200, 329, 687, 213, 283, 413}


In [48]:
if PROBLEM_SIZE <= 100:
    print('The following tyles were taken: ')
    for tile in current_state.taken:
        print(f'{tile}:\t', end='')
        for value in SETS[tile]:
            if value:
                print('1 ', end='')
            else:
                print('0 ', end='')
        print()

In [49]:
goal_check(current_state)

True